## Pryngles pixx external module

In [19]:
import pickle
import numpy as np

You must first download file `reflection-test.pkl` from Google Drive and place it at `/tmp`

In [3]:
f=open("/tmp/reflection-test.pkl","rb")
save_values=pickle.load(f)
f.close()

In [6]:
len(save_values)

362

In [25]:
for save in save_values:
    args=save["args"]
    """
    args are:
        0: Number of points (int)
        1: Delta azim (array Nx1)
        2: betas (array: Nx1)
        3: etas (array: Nx1)
        4: zetas
        5: nmugps
        6: nmatp
        7: nfoup
        8: xmup
        9: rfoup
        10: afp 
    """
    for arg in args:
        if isinstance(arg,np.ndarray):
            print(type(arg))
            print(arg.shape)
        else:
            print(arg)
    break

166
<class 'numpy.ndarray'>
(166,)
<class 'numpy.ndarray'>
(166,)
<class 'numpy.ndarray'>
(166,)
<class 'numpy.ndarray'>
(166,)
21
3
3
<class 'numpy.ndarray'>
(21,)
<class 'numpy.ndarray'>
(63, 21, 4)
<class 'numpy.ndarray'>
(166,)
